# Agente de IA sobre sentencias judiciales (RAG)

## Objetivo
Este notebook implementa un **agente de IA** cuya fuente de conocimiento es la base de datos vectorial de Pinecone creada con `indexar_sentencias_pinecone.ipynb`. La arquitectura es **RAG** (Retrieval-Augmented Generation):

1. El usuario hace una **pregunta** en lenguaje natural.
2. Se obtiene el **embedding** de la pregunta (OpenAI `text-embedding-3-small`).
3. Se **buscan** en Pinecone las sentencias más similares (por coseno).
4. Esos fragmentos se inyectan como **contexto** en un prompt para un modelo de lenguaje (OpenAI).
5. El modelo **responde** apoyándose solo en ese contexto, reduciendo alucinaciones.

## Requisitos
- Haber ejecutado antes `indexar_sentencias_pinecone.ipynb` para tener el índice `sentencias-judiciales` poblado.
- Variables de entorno en `.env`: `OPENAI_API_KEY` y `PINECONE_API_KEY`.

---
## 1. Imports y configuración

In [9]:
import os
import re
import textwrap
from dotenv import load_dotenv
from openai import OpenAI
from pinecone import Pinecone

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")

if not OPENAI_API_KEY:
    raise ValueError("Falta OPENAI_API_KEY en .env")
if not PINECONE_API_KEY:
    raise ValueError("Falta PINECONE_API_KEY en .env")

INDEX_NAME = "sentencias-judiciales"
EMBEDDING_MODEL = "text-embedding-3-small"
CHAT_MODEL = "gpt-4.1-mini"
TOP_K = 5

client_openai = OpenAI(api_key=OPENAI_API_KEY)
pc = Pinecone(api_key=PINECONE_API_KEY)
index = pc.Index(INDEX_NAME)

print(f"Conectado a índice '{INDEX_NAME}' y OpenAI ({CHAT_MODEL}).")

Conectado a índice 'sentencias-judiciales' y OpenAI (gpt-4.1-mini).


---
## 2. Búsqueda en la base vectorial

Función que convierte la pregunta en vector, consulta Pinecone y devuelve los metadatos y texto de las sentencias más relevantes.

**Consultas por Providencia:** Se reconocen formatos **T-388/19** (con guión), **A. 271/22** y **SU.174/21** (con punto, una o varias letras). Si el usuario menciona una o varias Providencias, se recuperan por metadatos (**fetch_by_metadata** con `$eq` o `$in`) probando variantes de formato (espacios, punto). Si no hay resultados, se hace búsqueda por vector y se filtra en Python.

In [10]:
# Patrones para Providencias (T-388/19, A. 271/22, SU.174/21)
PATRON_CON_GUION = re.compile(r"([A-Za-z]+)\s*-\s*(\d+)\s*/\s*(\d+)", re.IGNORECASE)
PATRON_CON_PUNTO = re.compile(r"([A-Za-z]+)\.\s*(\d+)\s*/\s*(\d+)", re.IGNORECASE)

def extraer_providencias(texto: str) -> list[str]:
    """Extrae TODAS las Providencias (ej: T-388/19, A. 271/22, SU.174/21). Lista sin duplicados."""
    if not texto or not texto.strip():
        return []
    t = texto.strip()
    out = []
    for m in PATRON_CON_GUION.finditer(t):
        letras, n1, n2 = m.group(1), m.group(2), m.group(3)
        out.append(f"{letras.upper()}-{n1}/{n2}")
    for m in PATRON_CON_PUNTO.finditer(t):
        letras, n1, n2 = m.group(1), m.group(2), m.group(3)
        out.append(f"{letras.upper()}. {n1}/{n2}")
    return list(dict.fromkeys(out))

def _variantes_valor(prov: str) -> list[str]:
    """Variantes por si en BD el formato difiere (espacios, punto)."""
    v = [prov, prov.replace("-", "- "), prov.replace("/", " / ")]
    if ". " in prov:
        v.append(prov.replace(". ", "."))
    return list(dict.fromkeys(v))

def _fetch_una_providencia(providencia: str, limit: int) -> list[dict]:
    """Recupera por una Providencia (probando variantes)."""
    for valor in _variantes_valor(providencia):
        try:
            resp = index.fetch_by_metadata(filter={"Providencia": {"$eq": valor}}, limit=limit)
        except Exception:
            continue
        if not getattr(resp, "vectors", None):
            continue
        out = [{"metadata": getattr(vec, "metadata", None) or {}, "score": None}
               for _id, vec in resp.vectors.items()]
        if out:
            return out[:limit]
    return []

def _fetch_por_providencias(providencias: list[str], limit: int) -> list[dict]:
    """Recupera por varias Providencias ($in o una a una)."""
    valores_in = list(providencias)
    for p in providencias:
        if ". " in p:
            valores_in.append(p.replace(". ", "."))
    valores_in = list(dict.fromkeys(valores_in))
    try:
        resp = index.fetch_by_metadata(filter={"Providencia": {"$in": valores_in}}, limit=limit)
        if getattr(resp, "vectors", None):
            out = [{"metadata": getattr(vec, "metadata", None) or {}, "score": None}
                   for _id, vec in resp.vectors.items()]
            if out:
                return out[:limit]
    except Exception:
        pass
    vistos = set()
    out = []
    for p in providencias:
        for r in _fetch_una_providencia(p, limit=limit):
            key = (r.get("metadata") or {}).get("Providencia", "") or id(r)
            if key not in vistos:
                vistos.add(key)
                out.append(r)
    return out[:limit]

def buscar_sentencias(pregunta: str, top_k: int = TOP_K) -> list[dict]:
    """
    Devuelve sentencias desde Pinecone. Si se mencionan Providencias (A. 271/22, T-388/19),
    se recuperan por metadatos (una o varias con $in/variantes) para no depender del vector.
    """
    listas = extraer_providencias(pregunta)

    if listas:
        out = _fetch_por_providencias(listas, limit=max(top_k, len(listas) * 2))
        if out:
            return out

    resp = client_openai.embeddings.create(
        model=EMBEDDING_MODEL,
        input=[pregunta.strip() or " "],
    )
    vector = resp.data[0].embedding
    results = index.query(vector=vector, top_k=top_k * 3, include_metadata=True)
    out = []
    for m in results.matches:
        meta = getattr(m, "metadata", None) or {}
        score = getattr(m, "score", None)
        if listas:
            prov_bd = str(meta.get("Providencia", "")).upper().replace(" ", "").replace(".", "").replace("/", "")
            if not any(p.upper().replace(" ", "").replace(".", "").replace("/", "") in prov_bd for p in listas):
                continue
        out.append({"metadata": meta, "score": score})
    return out[:top_k]

# Prueba rápida
prueba = buscar_sentencias("¿Qué dice la jurisprudencia sobre indemnización por despido?", top_k=2)
print(f"Recuperadas {len(prueba)} sentencias.")
if prueba:
    m = prueba[0]["metadata"]
    print(f"Ejemplo: Providencia = {m.get('Providencia', 'N/A')}, score = {prueba[0]['score']:.4f}")

Recuperadas 2 sentencias.
Ejemplo: Providencia = T-423/22, score = 0.5626


---
## 3. Construcción del contexto para el LLM

Se formatea el contenido recuperado (sintesis, tema, resuelve, providencia, fecha) en un único texto que el modelo usará como fuente.

In [11]:
def contexto_desde_resultados(resultados: list[dict]) -> str:
    """Convierte la lista de resultados de Pinecone en un bloque de texto para el prompt."""
    partes = []
    for i, r in enumerate(resultados, 1):
        meta = r.get("metadata") or {}
        prov = meta.get("Providencia", "")
        fecha = meta.get("Fecha Sentencia", "")
        tema = meta.get("Tema - subtema", "")
        resuelve = meta.get("resuelve", "")
        sintesis = meta.get("sintesis", "")
        bloque = f"--- Sentencia {i} (Providencia: {prov}, Fecha: {fecha}) ---\n"
        if tema:
            bloque += f"Tema: {tema}\n"
        if sintesis:
            bloque += f"Síntesis: {sintesis}\n"
        if resuelve:
            bloque += f"Resuelve: {resuelve[:1500]}..." if len(str(resuelve)) > 1500 else f"Resuelve: {resuelve}"
        partes.append(bloque)
    return "\n\n".join(partes) if partes else "(No se encontraron sentencias relevantes.)"

---
## 4. Agente RAG: pregunta → respuesta

Une búsqueda + contexto + llamada al chat de OpenAI con un system prompt que obliga al modelo a basarse solo en el contexto proporcionado.

In [12]:
SYSTEM_PROMPT = """Eres un asistente experto en jurisprudencia colombiana. Tu única fuente de información es el contexto que se te proporciona (fragmentos de sentencias). Responde de forma clara y concisa basándote únicamente en ese contexto. Si el contexto no contiene información suficiente para responder, dilo explícitamente. No inventes datos ni referencias."""

def preguntar(pregunta: str, top_k: int = TOP_K) -> str:
    """
    Flujo RAG: busca sentencias similares, arma el contexto y pide al LLM una respuesta.
    """
    resultados = buscar_sentencias(pregunta, top_k=top_k)
    contexto = contexto_desde_resultados(resultados)
    user_content = f"Contexto (sentencias recuperadas):\n\n{contexto}\n\n---\n\nPregunta del usuario: {pregunta}"
    resp = client_openai.chat.completions.create(
        model=CHAT_MODEL,
        messages=[
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": user_content},
        ],
        temperature=0.3,
    )
    return resp.choices[0].message.content or "(Sin respuesta)"

def imprimir_respuesta(pregunta: str, respuesta: str, ancho: int = 72):
    """Imprime pregunta y respuesta con texto replegado por párrafos para facilitar la lectura."""
    print("Pregunta:", pregunta)
    print()
    print("Respuesta del agente:")
    parrafos = respuesta.split("\n\n")
    for p in parrafos:
        p = p.strip()
        if p:
            print(textwrap.fill(p, width=ancho))
            print()

print("Función preguntar(pregunta) e imprimir_respuesta(...) listas. Ejemplo en la siguiente celda.")

Función preguntar(pregunta) e imprimir_respuesta(...) listas. Ejemplo en la siguiente celda.


---
## 5. Ejemplo de uso

Para cada caso hay dos tipos de consulta: **(1)** por la sentencia y **(2)** de qué tratan. La celda recorre todos los ejemplos con un `for` y muestra cada respuesta replegada a 72 caracteres.

**Caso 1 – Tres demandas (A. 271/22, A. 272/22, A. 273/22)**
- ¿Cuáles son las sentencias de las demandas A. 271/22, A. 272/22, A. 273/22?
- ¿De qué se trataron las 3 demandas anteriores?

**Caso 2 – Acoso escolar**
- ¿Cuál fue la sentencia del caso que habla de acoso escolar?
- ¿De qué trata la demanda relacionada con acoso escolar?

**Caso 3 – PIAR**
- ¿Existen casos que hablan sobre el PIAR? ¿Cuáles son sus sentencias?
- ¿De qué trataron los casos relacionados con el PIAR?

In [13]:
# Una consulta por cada sentencia/caso y otra "de qué tratan"
EJEMPLOS_PREGUNTAS = [
    # Caso 1 – Tres demandas
    "¿Cuáles son las sentencias de las demandas A. 271/22, C-173/20, T-330/24?",
    "¿De qué se trataron las 3 demandas anteriores?",
    # Caso 2 – Acoso escolar
    "¿Cuál fue la sentencia del caso que habla de acoso escolar?",
    "¿De qué trata la demanda relacionada con acoso escolar?",
    # Caso 3 – PIAR
    "¿Existen casos que hablan sobre el PIAR? ¿Cuáles son sus sentencias?",
    "¿De qué trataron los casos relacionados con el PIAR?",
]

# Iterar por cada ejemplo de pregunta
for i, pregunta in enumerate(EJEMPLOS_PREGUNTAS, 1):
    print(f"\n{'='*60}\nEjemplo {i} / {len(EJEMPLOS_PREGUNTAS)}\n{'='*60}")
    respuesta = preguntar(pregunta)
    imprimir_respuesta(pregunta, respuesta, ancho=72)
print("\n" + "="*60 + "\nFin de ejemplos.")


Ejemplo 1 / 6
Pregunta: ¿Cuáles son las sentencias de las demandas A. 271/22, C-173/20, T-330/24?

Respuesta del agente:
Las sentencias de las demandas son las siguientes:

1. Sentencia A. 271/22 (Fecha: 2022-03-09): Se rechazó la solicitud de
nulidad por falta de jurisdicción y competencia por el factor
territorial en un proceso de tutela relacionado con la Fundación
Matarife Internacional y las afirmaciones contra Álvaro Uribe Vélez.
Además, se negó la solicitud de notificación especial por medios
consulares y se ofició a la Fundación para que se pronuncie y aporte
soportes sobre las afirmaciones realizadas.

2. Sentencia C-173/20 (Fecha: 2020-06-10): Se declaró exequible el
Decreto Legislativo 545 de 2020, que suspendió temporalmente el
requisito de insinuación para algunas donaciones en el marco del Estado
de Emergencia Económica, Social y Ecológica derivado de la pandemia por
COVID-19, permitiendo agilizar donaciones destinadas a mitigar la crisis
sanitaria y económica.

3. Sente

---
## 6. Bucle interactivo

Ejecuta esta celda para hacer varias preguntas seguidas. Escribe `salir` o `quit` para terminar.

In [16]:
while True:
    pregunta = input("Tú: ").strip()
    if not pregunta or pregunta.lower() in ("salir", "quit", "exit"):
        print("Hasta luego.")
        break
    respuesta = preguntar(pregunta)
    print()
    imprimir_respuesta(pregunta, respuesta, ancho=72)
    print("-" * 60)


Pregunta: cual sentencia tiene tema COMPETENCIA DE LA JURISDICCIÓN CONTENCIOSO ADMINISTRATIVA-Controversias sobre recobros de prestaciones no incluidas en el plan de beneficios de salud-PBS

Respuesta del agente:
Las sentencias que tienen como tema "COMPETENCIA DE LA JURISDICCIÓN
CONTENCIOSO ADMINISTRATIVA-Controversias sobre recobros de prestaciones
no incluidas en el plan de beneficios de salud-PBS" son las siguientes:

1. Sentencia 1 (Providencia: A. 3082/23, Fecha: 2023-12-05) 2. Sentencia
2 (Providencia: A. 289/24, Fecha: 2024-02-14) 3. Sentencia 3
(Providencia: A. 2998/23, Fecha: 2023-11-28) 4. Sentencia 4
(Providencia: A. 2665/23, Fecha: 2023-10-25) 5. Sentencia 5
(Providencia: A. 2884/23, Fecha: 2023-11-21)

Todas estas sentencias abordan conflictos de competencia entre la
Jurisdicción Ordinaria en su especialidad laboral y la Jurisdicción de
lo Contencioso Administrativo sobre recobros de prestaciones no
incluidas en el plan de beneficios de salud (PBS).

--------------------